In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo2.csv")
dog_train=pd.read_csv("testDog2.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

print(dog_data)
print(dog_train)
print(kindCd_data)

      neuterYn sexCd  weight  noticeDays  age2  processState
0            N     F    7.46          10    12             0
1            N     M    7.00          14     1             1
2            U     M    4.50          11     2             0
3            N     M   10.00           8     1             0
4            N     M    6.00           8     4             0
...        ...   ...     ...         ...   ...           ...
22787        N     M    1.00          10     0             1
22788        N     M    1.00          10     0             1
22789        N     M    1.00          10     0             1
22790        U     F    6.00          12     3             0
22791        N     M    3.50          10     0             0

[22792 rows x 6 columns]
    neuterYn sexCd  weight  noticeDays  age2  processState
0          N     M     0.8          10     0             1
1          N     M     5.0          10     1             0
2          N     F     5.0          10    13             0
3     

In [65]:
# kindCd = np.array(kindCd_data, dtype = np.float64)
# kindCd_train=np.array(kindCd_data, dtype=np.float64)

# kindCd = kindCd.reshape(177)
# kindCd_train=kindCd_train.reshape(177)

# print(kindCd)
# print(kindCd.shape)

In [66]:
# # kindNum을 원핫 인코딩
# kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# # kindCd

# # 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_data.drop(['kindNum'], axis='columns', inplace=True)
# dog_data = pd.concat([dog_data, kindCd], axis=1)


# # kindNum을 원핫 인코딩
# kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# # 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_train.drop(['kindNum'], axis='columns', inplace=True)
# dog_train = pd.concat([dog_train, kindCd_train], axis=1)

# print(dog_data)
# print(dog_train)

In [67]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [74]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
# train_data = dummy_data(dog_train, dummy_columns)

print(data.head())
# print(train_data.head())

data = np.array(data, dtype = np.float64)
# train_data = np.array(train_data, dtype = np.float64)


from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), '훈련 샘플')
print(len(val), '검증 샘플')
print(len(test), '테스트 샘플')

   weight  noticeDays  age2  processState  neuterYn_N  neuterYn_U  neuterYn_Y  \
0    7.46          10    12             0           1           0           0   
1    7.00          14     1             1           1           0           0   
2    4.50          11     2             0           0           1           0   
3   10.00           8     1             0           1           0           0   
4    6.00           8     4             0           1           0           0   

   sexCd_F  sexCd_M  sexCd_Q  
0        1        0        0  
1        0        1        0  
2        0        1        0  
3        0        1        0  
4        0        1        0  
14586 훈련 샘플
3647 검증 샘플
4559 테스트 샘플


In [75]:
a = train[:, :3]
b = train[:, 4:]

# print(a[0,:])
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a = test[:, :3]
b = test[:, 4:]
testX = np.concatenate([a, b], axis = 1)

a = val[:, :3]
b = val[:, 4:]
valX = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape
testX.shape

[[ 1.  10.   0.  ...  0.   1.   0. ]
 [ 1.  10.   0.  ...  0.   1.   0. ]
 [14.  10.   3.  ...  1.   0.   0. ]
 ...
 [ 6.9 11.   4.  ...  1.   0.   0. ]
 [ 4.  10.   0.  ...  0.   1.   0. ]
 [ 6.4 10.   1.  ...  0.   1.   0. ]]


(4559, 9)

In [76]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=train[:,[3]]
testY = test[:,[3]]
valY = val[:,[3]]

# yData = yData.astype('int32')
print(yData)

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [0.]]


In [77]:
# 정규화

from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
xData = mm_scaler.fit_transform(xData)

In [78]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred)))

In [79]:
from sklearn.model_selection import GridSearchCV
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [3, 5, 8, 10, 12, 18],
           'min_samples_split' : [1, 2, 3, 5, 8, 10, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1, verbose = 1)
grid_cv.fit(xData, yData)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

Fitting 3 folds for each of 384 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:  2.6min finished
C:\Users\yunji\anaconda3\envs\dbswls\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 3, 'min_samples_split': 16, 'n_estimators': 100}
최고 예측 정확도: 0.6364


In [82]:
model = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 12,
                                min_samples_leaf = 3,
                                min_samples_split = 16,
                                random_state = 0,
                                n_jobs = -1)
rf_model=model.fit(xData, yData)
y_pred = rf_model.predict(testX)
model_performance(testY, y_pred)
# len(y_pred)

print('총 테스트 개수 : %d, 오류 개수 : %d' %(len(testY), (testY != y_pred).sum()))
print('정확도 : %.2f' %metrics.accuracy_score(testY, y_pred))

C:\Users\yunji\anaconda3\envs\dbswls\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


confusion matrix
[[1248 1283]
 [1047  981]]
accuracy : 0.488923009431893
precision : 0.4333038869257951
recall : 0.48372781065088755
F1 : 0.4571295433364399
총 테스트 개수 : 4559, 오류 개수 : 10384444
정확도 : 0.49


In [80]:
rf_model.predict_proba(testX[:6])

array([[0.        , 1.        ],
       [0.64494874, 0.35505126],
       [0.5       , 0.5       ],
       [0.44796212, 0.55203788],
       [0.90261905, 0.09738095],
       [0.90261905, 0.09738095]])

In [81]:
joblib.dump(rf_model, 'randomforest.pkl')

['randomforest.pkl']